#### 将前五种药物的推特都收集出来

In [3]:
import seaborn as sns
import numpy as np
import pandas as pd
from functools import partial
from multiprocessing import Pool
import multiprocessing
import time
import re
import pandas as pd
import tqdm 
out_dir = '/data1/data_8t/ivermectin/sentiment/' #保存文件的位置
data_type = {'created_at':'string','id':'string','full_text':'string','user':'object',"extracted_entity_sentiment":'object','drug_entity':'object',"sentiment":'object'}

In [4]:
#创建文件名列表
import glob
import pandas as pd
import tqdm   ## this is a ticker
data_dir = "/data1/data_8t/ivermectin/sentiment/drug+senti/" 
files = sorted(glob.glob(data_dir+"*.csv"))
file = '|'.join(files) #转为字符串

In [18]:
keys='vitamin d|Vitamin D2|Vitamin D3|vd3|vd2' #五种药物的keyword 以vd为例

def parallelize_dataframe(df, func, **kwargs):
    #CPUs = multiprocessing.cpu_count()
    num_partitions = 8 # number of partitions to split dataframe
    num_cores =  8 # number of cores on your machine

    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    func = partial(func, **kwargs)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df
    
def extract(df,keys):
    keys=keys
    
    # #关键词字符串
    # keywords = pd.read_csv('/data_8t/lwx/keywords/keywords.csv')
    # keys = keywords.Drugs.tolist()  #转为列表
    # keys = '|'.join(keys) #转为字符串

    #df = df[df['full_text'].str.contains(fr"\b(?:{keys})\b",case=False,flags=re.I)] #筛选包含keys的数据
    # flag=df['symp_entity'].str.contains(fr"\b(?:{keys})\b",case=False,flags=re.I)
    # diff_flag = [not f for f in flag]
    # res = df[diff_flag]
    # res=res.reset_index(drop=True)
    df = df[df['drug_entity'].str.contains(fr"\b(?:{keys})\b",case=False,flags=re.I)] #筛选包含keys的数据
    #print(len(df))
    df1 = df.drop_duplicates(subset=['id']) #去除重复项
    return df1


df_empty = pd.DataFrame(columns=["created_at","id","full_text","user",'drug_entity','sentiment'])
for file in tqdm.tqdm(files):
    df = pd.read_csv(file,dtype=data_type, lineterminator='\n',usecols=['created_at','id','full_text','user','drug_entity','sentiment'],low_memory=False, keep_default_na=False)#, iterator=True,chunksize=100000)
    df1=extract(df,keys)
    #df1 = parallelize_dataframe(df, extract)
    df_empty=pd.concat([df_empty,df1])
#keys='vitamin d|Vitamin D2|Vitamin D3|vd3|vd2|vit d|Calcifediol'
df_empty.to_csv(out_dir+keys+".csv",index=False)  #保存文件
print(len(df_empty))

100%|██████████| 135/135 [00:50<00:00,  2.68it/s]


86028


In [17]:
# 统计所有的积极消极中性情绪占比

import pandas as pd
symp=[]
df = pd.read_csv("/data1/data_8t/ivermectin/sentiment/zinc.csv",lineterminator='\n',low_memory=False, keep_default_na=False)
s=df['sentiment'].tolist()
symp.extend(s)
a=[]
for i in symp:
    b=(i[1:-1])
    if b!='':
       a.append((eval(b)))

b=[]
for i in a:
    if type(i)==tuple:
        i=list(i)
        for ii in i:
            #print(type(ii),ii)
            b.append(ii)
    else:
        #print(type(i),i)
        b.append(i)
dict = {}
for key in b:
    dict[key] = dict.get(key, 0) + 1
print(dict)

{'positive': 130206, 'neutral': 154257, 'negative': 3468}


#### 将每条推特中药物的情感个数提取出来

In [9]:
def Getsenti_num(twt):
    n=twt.count(',')+1
    return n

def Getpos_num(twt):
    n=twt.count('positive')
    return n

def Getneg_num(twt):
    n=twt.count('negative')
    return n

def Getneu_num(twt):
    n=twt.count('neutral')
    return n

def Senti_Entity(entity):#实体提取部分提取出drug实体
    senti_entity = []
    try:
        for i in eval(entity): 
            if i['type'] == 'Drug' and 'sentiment' in i:
                senti_entity.append(i['sentiment'].lower())
        return senti_entity
    except TypeError: # because some users has no text
        return '[]'

def parall_func(df):
    df['senti_num'] = df['sentiment'].apply(Getsenti_num)
    df['pos_num'] = df['sentiment'].apply(Getpos_num)
    df['neg_num'] = df['sentiment'].apply(Getneg_num)
    df['neu_num'] = df['sentiment'].apply(Getneu_num)
    return df


#pattern = re.compile('|'.join(symptom_keywords),flags=re.I)
# 这个函数就是用来做多线程的，输入是你要处理的数据框和相应的函数
def parallelize_dataframe(df, func, **kwargs):
    #CPUs = multiprocessing.cpu_count()
    num_partitions = 4 # number of partitions to split dataframe
    num_cores =  4 # number of cores on your machine

    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    func = partial(func, **kwargs)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

'''
# find the keywords in the text. if a keyword appears 
# several time in a text, only return one of it.
'''

'\n# find the keywords in the text. if a keyword appears \n# several time in a text, only return one of it.\n'

In [19]:
df = pd.read_csv('/data1/data_8t/ivermectin/sentiment/vitamin d|Vitamin D2|Vitamin D3|vd3|vd2.csv',lineterminator='\n',low_memory=False)
df = parallelize_dataframe(df, parall_func)
df.to_csv('/data1/data_8t/ivermectin/sentiment/vitamin d|Vitamin D2|Vitamin D3|vd3|vd2.csv')

In [19]:
#对于zinc 计算每周推文中positive的比例
import pandas as pd
df=pd.read_csv('zinc.csv')
data=df[['created_at','pos_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqzincp.csv')
#再除以每周实体总数

import pandas as pd
#df=pd.read_csv('zinc.csv')
data=df[['created_at','neg_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqzincn.csv')
#再除以每周实体总数

import pandas as pd
#df=pd.read_csv('zinc.csv')
data=df[['created_at','senti_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqzincs.csv')

/tmp/ipykernel_4168934/3533757849.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['created_at'] = pd.to_datetime(data['created_at'])
/tmp/ipykernel_4168934/3533757849.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['created_at'] = pd.to_datetime(data['created_at'])
/tmp/ipykernel_4168934/3533757849.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [6]:
# 对于HCQ
import pandas as pd
df=pd.read_csv('/data1/data_8t/ivermectin/sentiment/Hydroxychloroquine|HCQ|plaquenil.csv')
data=df[['created_at','pos_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqhcqp.csv')
#再除以每周实体总数

import pandas as pd
#df=pd.read_csv('hcq.csv')
data=df[['created_at','neg_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqhcqn.csv')
#再除以每周实体总数

import pandas as pd
#df=pd.read_csv('hcq.csv')
data=df[['created_at','senti_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqhcqs.csv')

/tmp/ipykernel_9324/1867587130.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['created_at'] = pd.to_datetime(data['created_at'])
/tmp/ipykernel_9324/1867587130.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['created_at'] = pd.to_datetime(data['created_at'])
/tmp/ipykernel_9324/1867587130.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [ ]:
# 对于zinc
import pandas as pd
df=pd.read_csv('zinc.csv')
data=df[['created_at','pos_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqzincp.csv')
#再除以每周实体总数

import pandas as pd
#df=pd.read_csv('zinc.csv')
data=df[['created_at','neg_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqzincn.csv')
#再除以每周实体总数

import pandas as pd
#df=pd.read_csv('zinc.csv')
data=df[['created_at','senti_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqzincs.csv')

In [11]:
# 对于remdesivir
import pandas as pd
df = pd.read_csv('/data1/data_8t/ivermectin/sentiment/Remdesivir|velkury.csv',lineterminator='\n',low_memory=False)
data=df[['created_at','pos_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqremp.csv')
#再除以每周实体总数

import pandas as pd
#df=pd.read_csv('rem.csv')
data=df[['created_at','neg_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqremn.csv')
#再除以每周实体总数

import pandas as pd
#df=pd.read_csv('rem.csv')
data=df[['created_at','senti_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqrems.csv')

/tmp/ipykernel_4168934/2750269058.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['created_at'] = pd.to_datetime(data['created_at'])
/tmp/ipykernel_4168934/2750269058.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['created_at'] = pd.to_datetime(data['created_at'])
/tmp/ipykernel_4168934/2750269058.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [ ]:
import pandas as pd
df=pd.read_csv('/data1/data_8t/ivermectin/sentiment/vitamin d|Vitamin D2|Vitamin D3|vd3|vd2.csv')
data=df[['created_at','pos_num']]
data.to_csv('vitd_senti.csv',index=False)

In [11]:
# 对vitd
data=pd.read_csv('vitd_senti.csv')
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqvitdp.csv')
#再除以每周实体总数

import pandas as pd
data=pd.read_csv('vitd_senti.csv')
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqvitdn.csv')
#再除以每周实体总数

import pandas as pd
data=pd.read_csv('vitd_senti.csv')
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqvitds.csv')

In [20]:
# 对ivermectin
import pandas as pd
df=pd.read_csv('/data1/data_8t/ivermectin/sentiment/ivermectin|Stromectol.csv')
data=df[['created_at','pos_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqiverp.csv')
#再除以每周实体总数

import pandas as pd
#df=pd.read_csv('iver.csv')
data=df[['created_at','neg_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqivern.csv')
#再除以每周实体总数

import pandas as pd
#df=pd.read_csv('iver.csv')
data=df[['created_at','senti_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqivers.csv')

/tmp/ipykernel_9324/3188282339.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['created_at'] = pd.to_datetime(data['created_at'])
/tmp/ipykernel_9324/3188282339.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['created_at'] = pd.to_datetime(data['created_at'])
/tmp/ipykernel_9324/3188282339.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [21]:
# 对vitd
import pandas as pd
df = pd.read_csv('/data1/data_8t/ivermectin/sentiment/vitamin d|Vitamin D2|Vitamin D3|vd3|vd2.csv',lineterminator='\n',low_memory=False)
df = parallelize_dataframe(df, parall_func)
df.to_csv('/data1/data_8t/ivermectin/sentiment/vitamin d|Vitamin D2|Vitamin D3|vd3|vd2.csv',index=False)
data=df[['created_at','pos_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqvitdp.csv')
#再除以每周实体总数

import pandas as pd
#df=pd.read_csv('iver.csv')
data=df[['created_at','neg_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqvitdn.csv')
#再除以每周实体总数

import pandas as pd
#df=pd.read_csv('iver.csv')
data=df[['created_at','senti_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqvitds.csv')

/tmp/ipykernel_9324/3129157154.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['created_at'] = pd.to_datetime(data['created_at'])
/tmp/ipykernel_9324/3129157154.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['created_at'] = pd.to_datetime(data['created_at'])
/tmp/ipykernel_9324/3129157154.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://